In [ ]:
#Weighted Model With 80% Daily and 20% Climate change
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime
import numpy as np

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Automatically get current date in the required format
current_date = datetime.now().strftime("%Y-%m-%d")

# Get user input for latitude and longitude
latitude = float(input("Enter Latitude: "))
longitude = float(input("Enter Longitude: "))

# Make sure all required weather variables are listed here
url = "https://my-server.tld/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location for current weather data
response = responses[0]
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

# Calculate daily average temperature (mean of max and min)
daily_temperature_avg = (daily_temperature_2m_max + daily_temperature_2m_min) / 2

# Set up data for the climate change model
climate_url = "https://climate-api.open-meteo.com/v1/climate"
params_climate = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": current_date,
    "end_date": current_date,
    "models": ["CMCC_CM2_VHR4", "FGOALS_f3_H", "HiRAM_SIT_HR", "MRI_AGCM3_2_S", "EC_Earth3P_HR", "MPI_ESM1_2_XR", "NICAM16_8S"],
    "disable_bias_correction": True,
    "daily": ["temperature_2m_mean", "precipitation_sum"]
}
responses_climate = openmeteo.weather_api(climate_url, params=params_climate)

# Process climate change model data
response_climate = responses_climate[0]
daily_climate = response_climate.Daily()

# Get values for temperature and precipitation from all models
daily_temperature_2m_mean_models = np.array([daily_climate.Variables(i).ValuesAsNumpy() for i in range(7)])  # 7 models
daily_precipitation_sum_models = np.array([daily_climate.Variables(i + 7).ValuesAsNumpy() for i in range(7)])  # 7 models

# Average the values across the models (mean temperature and precipitation)
daily_temperature_2m_mean_avg = daily_temperature_2m_mean_models.mean(axis=0)
daily_precipitation_sum_avg = daily_precipitation_sum_models.mean(axis=0)

# Combine the daily data and climate model data with a weighted average
temperature_weighted = 0.6 * daily_temperature_avg + 0.4 * daily_temperature_2m_mean_avg
precipitation_weighted = 0.6 * daily_precipitation_sum + 0.4 * daily_precipitation_sum_avg

# Create a DataFrame with the combined data
daily_data = {
    "date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    ),
    "temperature_avg_weighted": temperature_weighted,
    "precipitation_sum_weighted": precipitation_weighted
}

daily_dataframe = pd.DataFrame(data=daily_data)
print(daily_dataframe)

